# Libraries

In [1]:
import gradio as gr
import pandas as pd
import numpy as np
import json
import io 
import sys
from PIL import Image
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS

c:\Users\bpadmin\anaconda3\envs\tf_tr_recommender\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# vector db creation

In [2]:
# encoded_stock_data_path = '../data/sl20.xlsx'
# # stock_data = pd.read_csv(encoded_stock_data_path)
# stock_data = pd.read_excel(encoded_stock_data_path)
# stock_data.head(2)
# stock_data = stock_data.fillna('unknown')
# stock_data['symbol'] = stock_data['symbol'].str.replace(r"(","")
# stock_data['symbol'] = stock_data['symbol'].str.replace(r")","")
# embeddings = HuggingFaceEmbeddings()
# stock_meta_data = [{'symbol': stock_data.iloc[idx]['symbol'], 'name' : stock_data.iloc[idx]['name']} for idx in range(stock_data.shape[0])]
# stock_meta_data
# originals = [ stock_data.iloc[idx]['name'] + ' ' + stock_data.iloc[idx]['buisnesssummary'] for idx in range(stock_data.shape[0])]
# originals[:5]
# from langchain_community.vectorstores import FAISS
# vectors = embeddings.embed_documents(originals)
# '''Tuple[str, List[float]]'''

# stock_data_embeddings = [(originals[idx],vectors[idx]) for idx in range(len(vectors))]
# db = FAISS.from_embeddings(text_embeddings = stock_data_embeddings, embedding = embeddings, metadatas= stock_meta_data)
# db.save_local("../vector_db/faiss_sl20_index")


# Recommendation generation

In [3]:
embeddings = HuggingFaceEmbeddings()

In [4]:
new_db = FAISS.load_local("../vector_db/faiss_sl20_index", embeddings)

In [5]:
def get_recommendations(query, top_k):
  relevant_stocks = new_db.similarity_search_with_relevance_scores(query= query, k = top_k)
  recommendations = [(relevant_stocks[idx][0].metadata.get('symbol'), relevant_stocks[idx][0].metadata.get('name')) for idx in range(len(relevant_stocks))]
  for idx, data in enumerate(recommendations):
    print('{}. {} : {}'.format(idx+1, data[0], data[1]))


In [6]:
stock_pref = 'businesses in the domain of shipping and logistics'

get_recommendations(stock_pref, top_k = 10)

1. JKH : JOHN KEELLS HOLDINGS PLC
2. LOLC : L O L C HOLDINGS PLC
3. LIOC : LANKA IOC PLC
4. COMB : COMMERCIAL BANK OF CEYLON PLC
5. NTB : NATIONS TRUST BANK PLC
6. CIC : C I C HOLDINGS PLC
7. SAMP : SAMPATH BANK PLC
8. HAYL : HAYLEYS PLC
9. HHL : HEMAS HOLDINGS PLC
10. LOFC : LOLC FINANCE PLC


# Insight generation

In [7]:
from datetime import datetime

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [8]:
data_path = '../data/SL20 Two years to Narada.xls'

data = pd.read_excel(data_path, index_col= False)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [9]:
def split_(stock_name):
    return stock_name.split('.')[0]

In [10]:
data['SECURITYCODE'] = data['SECURITYCODE'].apply(lambda x : split_(x))

In [11]:
data = data[['SECURITYCODE','OPENINGPRICE','HIGHPX', 'LOWPX', 'CLOSINGPRICE', 'TRADEDATE', 'UNIX_TS']].dropna(axis= 0, how = 'any')
# data

In [12]:
def plot_ohlc(data, title):
    x_vals = data.TRADEDATE

    fig = make_subplots(rows=1, cols=1, vertical_spacing=0.01 , specs=[[{'rowspan':1, 'type':'Candlestick'}]],shared_xaxes=True)
    ohlc_obj = go.Candlestick(x=x_vals, open = data.OPENINGPRICE, high=data.HIGHPX, low=data.LOWPX, close=data.CLOSINGPRICE, name= title)
    fig.add_trace(ohlc_obj, row = 1, col = 1)
    fig.update_layout(title = title)
    fig.show(config={
        'modeBarButtonsToRemove': ['zoom', 'pan']
    })

In [31]:
from turtle import bgcolor
from matplotlib.axis import XAxis


def plot_ohlc_w_des(data, titile, des):
    x_vals = data.TRADEDATE

    fig = make_subplots(rows=1, cols=2, vertical_spacing=0.01, column_widths=[0.15, 0.85], subplot_titles=["", "Chart"]) #,shared_xaxes=True
    ohlc_obj = go.Candlestick(x=x_vals, open = data.OPENINGPRICE, high=data.HIGHPX, low=data.LOWPX, close=data.CLOSINGPRICE, name= 'OHLC')
    fig.add_trace(ohlc_obj, row = 1, col = 2)
    fig.add_trace(go.Scatter(x=[], y=[]), row=1, col=1)

    fig.add_annotation(x=10, y=10, xref="paper", yref="paper",
                   text=des, showarrow=False,
                   align = 'left',
                   font=dict(size=10, color="black"),
                   row=1, col=1)

    fig.update_layout(xaxis=dict(visible=False), yaxis=dict(visible=False))
    fig.update_yaxes(showgrid=False, zeroline=False, showticklabels=False, row=1, col=1)
    fig.update_annotations(selector=dict(row=1, col=1), paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')
    fig.update_layout(height=500, width=1500, title_text=titile)

    return fig

## OHLC candles

## Volatility calculations

In [14]:
def get_log_returns_v2(df ,close_col , horizon):
  data1 = df[close_col][horizon:].to_numpy()
  data2 = df[close_col][:-horizon].to_numpy()
  df = df.reset_index(drop = True)
  df.loc[horizon:, 'returns'] = data1/data2
  df['log_returns'] = np.log(df['returns'])
  df = df.dropna(axis = 0, how = 'any').reset_index(drop = True)
  return df

In [15]:
def get_std_volatility(df, close_col = 'CLOSINGPRICE', horizon = 5):

    df_out = get_log_returns_v2(df, 'CLOSINGPRICE', 5)
    return_mean = df_out.log_returns.mean()
    return_std = df_out.log_returns.std()

    return return_mean, return_std

### Garman Klass volatility
![image.png](attachment:image.png)

In [16]:
def GKHV(o,h,l,c):

    volatility = (1/2)*((np.log(h/l))**2) + (2*(np.log(2))-1)*((np.log(c/o))**2)

    return volatility

### Rogers and Satchell

![image.png](attachment:image.png)

In [17]:
def RS(o,h,l,c,prev_c):

    u = np.log(h/o)
    c = np.log(c/o)
    d = np.log(l/o)


    volatility = u*(u-c) + d*(d-c)

    return volatility


### Yand and Zhang
![image.png](attachment:image.png)

In [18]:
symbol_volatilities = pd.DataFrame()
v_dict = {}
symbols = data
for symbol in symbols:
    # print(symbol)
    v_dict['symbol'] = symbol

    symbol_df = data.loc[data.SECURITYCODE == symbol].sort_values(by = 'UNIX_TS').reset_index(drop = True)
    
    symbol_df['gkhv'] = symbol_df.apply(lambda row: GKHV(row['OPENINGPRICE'],row['HIGHPX'],row['LOWPX'],row['CLOSINGPRICE']), axis = 1)
    gkhv = np.sqrt(symbol_df.gkhv.mean())
    v_dict['gkhv'] = gkhv

    symbol_df['prev_c'] = symbol_df['CLOSINGPRICE'].shift(1)
    symbol_df['rs'] = symbol_df.apply(lambda row: RS(row['OPENINGPRICE'],row['HIGHPX'],row['LOWPX'],row['CLOSINGPRICE'], row['prev_c']), axis = 1)
    rs = np.sqrt(symbol_df.rs.mean())
    v_dict['rs'] = rs

    symbol_df['norm_o'] = symbol_df['OPENINGPRICE']/symbol_df['CLOSINGPRICE'].shift(1)
    symbol_df['norm_c'] = symbol_df['CLOSINGPRICE']/symbol_df['OPENINGPRICE']
    k = (0.34/(1.34 + ((len(symbol_df)+1)/(len(symbol_df)-1))))
    yangzhang = np.sqrt((symbol_df['norm_o'].std()**2) + (k*symbol_df['norm_c'].std()**2) + (1-k)*symbol_df['rs'].mean())
    v_dict['yangzhang'] = yangzhang

    symbol_volatilities = pd.concat([symbol_volatilities, pd.DataFrame([v_dict])], ignore_index = True)

### get_volatlity_insights

In [19]:
def get_volatility_insights(symbol_df):

    symbol_df['gkhv'] = symbol_df.apply(lambda row: GKHV(row['OPENINGPRICE'],row['HIGHPX'],row['LOWPX'],row['CLOSINGPRICE']), axis = 1)
    gkhv = (np.sqrt(symbol_df.gkhv.mean()))*100

    symbol_df['prev_c'] = symbol_df['CLOSINGPRICE'].shift(1)
    symbol_df['rs'] = symbol_df.apply(lambda row: RS(row['OPENINGPRICE'],row['HIGHPX'],row['LOWPX'],row['CLOSINGPRICE'], row['prev_c']), axis = 1)
    rs = (np.sqrt(symbol_df.rs.mean()))*100

    symbol_df['norm_o'] = symbol_df['OPENINGPRICE']/symbol_df['CLOSINGPRICE'].shift(1)
    symbol_df['norm_c'] = symbol_df['CLOSINGPRICE']/symbol_df['OPENINGPRICE']
    k = (0.34/(1.34 + ((len(symbol_df)+1)/(len(symbol_df)-1))))
    
    yangzhang = np.sqrt((symbol_df['norm_o'].std()**2) + (k*symbol_df['norm_c'].std()**2) + (1-k)*symbol_df['rs'].mean())*100
    
    return gkhv, rs, yangzhang

## Drowdown

In [20]:
# returns percentage
def max_drowdown(symb_df,close):
    max_prs = close.rolling(window = len(symb_df), min_periods = 1).max()
    dd = (close/max_prs) - 1
    max_dd = dd.rolling(window = len(dd), min_periods=1).min()
    return max_dd.min()*100


## Returns

In [21]:
def find_act_fday(symbol_df ,possible_dates):

    for pos_date in possible_dates:
    # print(pos_date)
        if pos_date in symbol_df.TRADEDATE.values:
            break
    return pos_date

### get_yr_2_date

In [36]:
#returns the percentage value
def get_yr_2_date(symbol_df):
    fday = pd.Timestamp(datetime((pd.Timestamp.today().year),1,1).date())
    possible_dates = pd.bdate_range(fday, fday+pd.offsets.BusinessDay(n=5))

    act_fday = find_act_fday(symbol_df,possible_dates)
    
    fday_value = symbol_df.loc[symbol_df.TRADEDATE == act_fday].CLOSINGPRICE.iloc[0]
    today_value = symbol_df.iloc[-1].CLOSINGPRICE

    yr_2_dt_return = (today_value - fday_value)/fday_value

    yr_2_date_df = symbol_df[symbol_df.TRADEDATE >= act_fday].sort_values(by = 'UNIX_TS')
    gkhv, rs, yangzhang = get_volatility_insights(yr_2_date_df)

    max_dd = max_drowdown(yr_2_date_df, yr_2_date_df['CLOSINGPRICE'])

    # return yr_2_dt_return*100, max_dd, gkhv, rs, yangzhang
    return round(yr_2_dt_return*100, 2), round(max_dd, 2) , round(gkhv, 2), round(rs, 2), round(yangzhang, 2)

### get_yr

In [35]:
# returns the percentage value
def get_yr(symbol_df):

    fday = symbol_df.TRADEDATE.iloc[-1] - pd.DateOffset(years=1)
    possible_dates = pd.bdate_range(fday, fday+pd.offsets.BusinessDay(n=5))

    act_fday = find_act_fday(symbol_df,possible_dates)
    
    fday_value = symbol_df.loc[symbol_df.TRADEDATE == act_fday].CLOSINGPRICE.iloc[0]
    today_value = symbol_df.iloc[-1].CLOSINGPRICE

    yr_return = (today_value - fday_value)/fday_value

    yr_df = symbol_df[symbol_df.TRADEDATE >= act_fday].sort_values(by = 'UNIX_TS')
    gkhv, rs, yangzhang = get_volatility_insights(yr_df)

    max_dd = max_drowdown(yr_df, yr_df['CLOSINGPRICE'])

    # return yr_return*100, max_dd, gkhv, rs, yangzhang
    return round(yr_return*100, 2), round(max_dd, 2) , round(gkhv, 2), round(rs, 2), round(yangzhang, 2)

### get_mn

In [24]:
# returns the percentage value
def get_mn(symbol_df):

    fday = symbol_df.TRADEDATE.iloc[-1] - pd.DateOffset(months=1)
    possible_dates = pd.bdate_range(fday, fday+pd.offsets.BusinessDay(n=5))

    act_fday = find_act_fday(symbol_df,possible_dates)
    
    fday_value = symbol_df.loc[symbol_df.TRADEDATE == act_fday].CLOSINGPRICE.iloc[0]
    today_value = symbol_df.iloc[-1].CLOSINGPRICE

    mn_return = (today_value - fday_value)/fday_value

    mn_df = symbol_df[symbol_df.TRADEDATE >= act_fday].sort_values(by = 'UNIX_TS')
    gkhv, rs, yangzhang = get_volatility_insights(mn_df)

    max_dd = max_drowdown(mn_df, mn_df['CLOSINGPRICE'])

    return round(mn_return*100, 2), round(max_dd, 2) , round(gkhv, 2), round(rs, 2), round(yangzhang, 2)

In [25]:
def get_insights(symbol):
    insight_dict = {'year_to_date':{}, 'last_year':{}, 'last_month' : {}}
    symbol_df = data.loc[data.SECURITYCODE == symbol].sort_values(by = 'UNIX_TS')

    insight_dict['year_to_date']['yr_2_dt_return'], insight_dict['year_to_date']['yr_2_dt_mx_dd'], insight_dict['year_to_date']['gkhv'], insight_dict['year_to_date']['rs'], insight_dict['year_to_date']['yangzhang'] = get_yr_2_date(symbol_df)
    insight_dict['last_year']['yr_2_dt_return'], insight_dict['last_year']['yr_2_dt_mx_dd'], insight_dict['last_year']['gkhv'], insight_dict['last_year']['rs'], insight_dict['last_year']['yangzhang'] = get_yr(symbol_df)
    insight_dict['last_month']['yr_2_dt_return'], insight_dict['last_month']['yr_2_dt_mx_dd'], insight_dict['last_month']['gkhv'], insight_dict['last_month']['rs'], insight_dict['last_month']['yangzhang'] = get_mn(symbol_df)

    return insight_dict

In [26]:
def get_insights_html(symbol):
    insight_dict = {'year_to_date':{}, 'last_year':{}, 'last_month' : {}}
    symbol_df = data.loc[data.SECURITYCODE == symbol].sort_values(by = 'UNIX_TS')

    y2d = get_yr_2_date(symbol_df)
    yr = get_yr(symbol_df)
    mn = get_mn(symbol_df)

    desc = 'year to date : <br>'\
    '   return : {}%<br>'\
    '   max drow down : {}%<br>'\
    '   gkhv : {}%<br>'\
    '   rs : {}%<br>'\
    '   yangzhang : {}%<br>'\
    ' <br>'\
    'last year : <br>'\
    '   return : {}%<br>'\
    '   max drow down : {}%<br>'\
    '   gkhv : {}%<br>'\
    '   rs : {}%<br>'\
    '   yangzhang : {}%<br>'\
    ' <br>'\
    'last month : <br>'\
    '   return : {}%<br>'\
    '   max drow down : {}%<br>'\
    '   gkhv : {}%<br>'\
    '   rs : {}%<br>'\
    '   yangzhang : {}%<br>'.format(y2d[0],y2d[1],y2d[2],y2d[3],y2d[4],
                        yr[0],yr[1],yr[2],yr[3],yr[4],
                        mn[0],mn[1],mn[2],mn[3],mn[4])
    return desc

# Combined Recommender

In [27]:
price_data = data.copy()
price_data.head(2)

,SECURITYCODE,OPENINGPRICE,HIGHPX,LOWPX,CLOSINGPRICE,TRADEDATE,UNIX_TS
1,VONE,38.5,39.2,38.4,38.9,2024-02-07,1707264000
2,NDB,64.9,64.9,63.9,63.9,2024-02-07,1707264000


In [34]:
def quey_generator(domains):
    query = 'business that belong in the domains of '+','.join(domains)
    return query

def get_recommendations_gradio(domains, top_k = 5):
  query = quey_generator(domains)
  relevant_stocks = new_db.similarity_search_with_relevance_scores(query= query, k = top_k)
  recommendations = [(relevant_stocks[idx][0].metadata.get('symbol'), relevant_stocks[idx][0].metadata.get('name')) for idx in range(len(relevant_stocks))]

  recommended_symbols = []
  reco_images = []
  for idx, data in enumerate(recommendations):
    recommended_symbols.append(data[0])
    symbol_df = price_data[price_data.SECURITYCODE == data[0]].sort_values(by = 'UNIX_TS').reset_index(drop = True)
    reco_img = plot_ohlc_w_des(symbol_df, '{} : {}'.format(data[0], data[1]) ,get_insights_html(data[0]))
    reco_images.append(reco_img)
  return reco_images

with gr.Blocks() as demo:
    inputs = gr.Dropdown(
        ["finance", "transport", "technology", "agriculture"],
        multiselect=True,
        label="Domains",
        allow_custom_value=True,
        scale=5
    )
    rec1, rec2, rec3, rec4, rec5 = [gr.Plot(), gr.Plot(),gr.Plot(),gr.Plot(),gr.Plot()]

    inputs.change(get_recommendations_gradio, inputs=inputs, outputs=[rec1, rec2, rec3, rec4, rec5])
    demo.load(get_recommendations_gradio, inputs=[inputs], outputs=[rec1, rec2, rec3, rec4, rec5])
    demo.launch(share = True)

Running on local URL:  http://127.0.0.1:7864
Running on public URL: https://ca18d010ac9fa58f61.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [29]:
sys.exit()

SystemExit: 

c:\Users\bpadmin\anaconda3\envs\tf_tr_recommender\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Sandbox

In [ ]:
def plotly_fig2array(fig):
    #convert Plotly fig to  an array
    fig_bytes = fig.to_image(format="png")
    buf = io.BytesIO(fig_bytes)
    img = Image.open(buf)
    return np.asarray(img)

def quey_generator(domains):
    query = 'business that belong in the domains of '+','.join(domains)
    return query

def get_recommendations_gradio(domains, top_k = 5):
  query = quey_generator(domains)
  relevant_stocks = new_db.similarity_search_with_relevance_scores(query= query, k = top_k)
  recommendations = [(relevant_stocks[idx][0].metadata.get('symbol'), relevant_stocks[idx][0].metadata.get('name')) for idx in range(len(relevant_stocks))]

  recommended_symbols = []
  reco_images = []
  for idx, data in enumerate(recommendations):
    recommended_symbols.append(data[0])
    symbol_df = price_data[price_data.SECURITYCODE == data[0]].sort_values(by = 'UNIX_TS').reset_index(drop = True)
    reco_img = plot_ohlc_w_des(symbol_df, data[0] ,get_insights_html(data[0]))
    # reco_images.append(plotly_fig2array(reco_img))
    reco_images.append(reco_img)
  return reco_images


demo = gr.Interface(
    fn=get_recommendations_gradio,
    inputs=gr.Dropdown(
        ["finance", "transport", "technology", "agriculture"],
        multiselect=True,
        label="Domains",
        allow_custom_value=True,
        scale=5
    ),
    outputs=[gr.Plot(min_width= 1000),gr.Plot(min_width= 1000),gr.Plot(min_width= 1000),gr.Plot(min_width= 1000),gr.Plot(min_width= 1000)],  #gr.Textbox(label="Description"), gr.Label("Recommendations"), 
)

# Launch the interface
# demo.launch(share=True)

In [ ]:
test_plot = get_recommendations_gradio(['financial'])[0]

In [ ]:
with gr.Blocks() as demo:
    gr.Plot(value=test_plot)

demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [ ]:
import plotly.io as pio
pio.write_image(test_plot, 'test_plot_save', format='png')

# test_plot.write_image('test_plot_save')

In [ ]:
# data[data.SECURITYCODE == 'VONE'].sort_values(by = 'UNIX_TS').reset_index(drop = True)

In [ ]:
def get_recommendations(query, top_k, price_data):
  relevant_stocks = new_db.similarity_search_with_relevance_scores(query= query, k = top_k)
  recommendations = [(relevant_stocks[idx][0].metadata.get('symbol'), relevant_stocks[idx][0].metadata.get('name')) for idx in range(len(relevant_stocks))]
  for idx, data in enumerate(recommendations):
    symbol_df = price_data[price_data.SECURITYCODE == data[0]].sort_values(by = 'UNIX_TS').reset_index(drop = True)
    print('{}. {} : {}'.format(idx+1, data[0], data[1]))
    print()
    print('Insights :')
    # for key, value in get_insights(data[0]).items():
    #   print(key + ': ' + str(value))
  
    print(json.dumps(get_insights(data[0]), indent=4))
    print()
    plot_ohlc(symbol_df, data[0])



In [ ]:
def get_recommendations_imgs(query, top_k, price_data):
  relevant_stocks = new_db.similarity_search_with_relevance_scores(query= query, k = top_k)
  recommendations = [(relevant_stocks[idx][0].metadata.get('symbol'), relevant_stocks[idx][0].metadata.get('name')) for idx in range(len(relevant_stocks))]
  reco_images = []

  for idx, data in enumerate(recommendations):
    symbol_df = price_data[price_data.SECURITYCODE == data[0]].sort_values(by = 'UNIX_TS').reset_index(drop = True)
    insights_json = json.dumps(get_insights(data[0]), indent=4)
    print('{}. {} : {}'.format(idx+1, data[0], data[1]))

    reco_img = plot_ohlc_w_des(symbol_df, data[0] ,get_insights_html(data[0]))
    reco_images.appendz(reco_images)

  return reco_images



In [ ]:
stock_pref = 'businesses in the domain of shipping and logistics'

get_recommendations_imgs(stock_pref, top_k = 5, price_data = data)


In [ ]:
def quey_generator(domains):
    query = 'business that belong in the domains of '+','.join(domains)
    return query

def get_recommendations_gradio(domains, top_k = 5):
  query = quey_generator(domains)
  relevant_stocks = new_db.similarity_search_with_relevance_scores(query= query, k = top_k)
  recommendations = [(relevant_stocks[idx][0].metadata.get('symbol'), relevant_stocks[idx][0].metadata.get('name')) for idx in range(len(relevant_stocks))]

  reco_list = []
  recommended_symbols = []
  for idx, data in enumerate(recommendations):
    recommended_symbols.append(data[0])
    reco_list.append('{}. {} : {}'.format(idx+1, data[0], data[1]))
    
  # return_str = '\n'.join(reco_list)
  radio_bts = [(get_insights_html(symbol), symbol) for symbol in recommended_symbols]

  html_recos = '\n'.join(reco_list)
  
  # print(radio_bts)
  return radio_bts

In [ ]:
import gradio as gr

# Define the query generator function
def query_generator(domains):
    query = 'businesses that belong in the domains of ' + ', '.join(domains)
    return query

# Define the function to get recommendations
def get_recommendations_gradio(domains, top_k=5):
    # Generate query based on selected domains
    query = query_generator(domains)
    
    # Get relevant stocks based on the query
    relevant_stocks = new_db.similarity_search_with_relevance_scores(query=query, k=top_k)
    
    # Extract recommendations (symbols and names)
    recommendations = [(relevant_stocks[idx][0].metadata.get('symbol'), relevant_stocks[idx][0].metadata.get('name')) for idx in range(len(relevant_stocks))]

    reco_list = []
    recommended_symbols = []
    for idx, data in enumerate(recommendations):
        recommended_symbols.append(data[0])
        reco_list.append('{}. {} : {}'.format(idx + 1, data[0], data[1]))

    # print(recommended_symbols)
    # Create radio buttons along with associated symbols and insights
    return_radio_bts = [(get_insights_html(symbol), symbol) for symbol in recommended_symbols]
    print(return_radio_bts)
    # Add description below the plot
    description = "This plot shows recommendations based on selected domains."
    
    return return_radio_bts #description, reco_list, 

# Create Gradio interface
demo = gr.Interface(
    fn=get_recommendations_gradio,
    inputs=gr.Dropdown(
        ["finance", "transport", "technology", "agriculture"],
        multiselect=True,
        label="Domains",
        allow_custom_value=True,
        scale=5
    ),
    outputs=gr.Radio(),  #gr.Textbox(label="Description"), gr.Label("Recommendations"), 
)

# Launch the interface
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
